# Note: 
### The code below runs a single simulation of the Nipah model. 

In [1]:
#This is to avoid deprecation warnings with Shapely / Pysal
import warnings 
warnings.filterwarnings('ignore')

In [42]:
#Set the working directory to a folder where all the files are located
import os
os.chdir('Path/to/your/folder/')

In [3]:
#Import all packages needed for the simulation to run
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import truncnorm
import geopandas as gpd
from uuid import uuid4
import pysal as ps

## Set up geographical components of the model and determine the emergence location

In [4]:
## Create a function that will be used to determine where the pathogen will emerge in the simulation 
#This function normalizes the risk so that the sum of all risks is 1 and adds the normalized result to the geodataframe
#Finally it draws one district using a multinomial

#Function takes one argument:
#gdf - geo dataframe as per geopandas - usually imported from a shapefile
def emergence_location(gdf):
    gdf['Std_bats'] = gdf['BATS'] / np.sum(gdf['BATS']) #normalize the measure and add to geo dataframe

    emerg_location_index, = np.where(np.random.multinomial(1, gdf['Std_bats']) == 1)[0] #draw an index for the district where epidemic will be seeded

    return emerg_location_index #return the index value


In [43]:
#Import the shapefile with the geography of India and Bangladesh 
#The geodataframe also contains the information on bat species used to determine the emergence risk
file = 'batpoly.shp'
gdf = gpd.read_file(file)

#Get a queen contiguity weight, used to figure out neighbors
#IMPORTANT NOTE: This step only works with shapely < 2.0 
w = ps.lib.weights.contiguity.Queen.from_dataframe(gdf, silence_warnings = True)

#Import gravity model matrix for connectivity
connectivity = pd.read_csv('Gravity_mod_high.csv')
connectivity = connectivity.set_index('From')

#Determine the district in which the outbreak will be seeded using the function created above 
#Note that this is an index value
emerg_loc = emergence_location(gdf)

## Set the parameters for the simulation

In [22]:
#Transmission
beta = 0.4

#Unsafe contact rate for healthcare workers
unsafe = 0.05

#Vaccination rate of HCWs - 1 / days
vacc_rate_HCW = 1 / 3

#Ring vaccination rate - 1 / days
ring_vac = 1/10

#Random vaccination mean number per infected
#This corresponds to an average of missed contacts for each infected
random_vac = 3
random_vac_std = 5

#truncated normal function to draw that number of individuals
#Truncated normal function to draw a number of individuals per family/household
tn_vac = truncnorm((0-random_vac)/random_vac_std, (1000-random_vac)/random_vac_std, loc = random_vac, scale = random_vac_std)

#Wait time for contact tracing
days_random_vac = 7

#Average family/household size
#Also std dev of the family/household size
#Parameterized as the number of contacts based on Gurley et al. 2007 Emerging Infectious Diseases
tau = 20
tau_std = 16

#Truncated normal function to draw a number of individuals per family/household
tn = truncnorm((0-tau)/tau_std, (1000-tau)/tau_std, loc = tau, scale = tau_std)

#Vaccine efficacy
phi = 0.75

#Duration of exposed period (1 / days)
omega = 1/8

#Duration of infection before recovery (1 / days)
gamma = 1/10

#Death rate
alpha = 3/10

#Average time for an infected individual in the hospital to get to the hospital
pH_mean = 3
#Time for an infected individual in the family layer to get to the hospital
#This varies by district
time_to_hospital = pd.DataFrame({'location' : gdf['adm2'], 'healthcare' : pH_mean * (1 - gdf['Healthcare']) / np.mean(1 - gdf['Healthcare'])})
time_to_hospital['healthcare'] = np.where(time_to_hospital['healthcare'] < 0, 0.005, time_to_hospital['healthcare'])
time_to_hospital = time_to_hospital.set_index('location')

#Probability of an hospitalized recovered to be returned in the community
pC = 1/2

#Threshold for vaccination (number of cases)
#This is to be understood per admin2 district
threshold = 1

#Number of days after last case to stop the simulation
min_time_after_epidemic = 10

#Delay between reaching threshold and vaccination start  (in days)
vacc_delay = 7

#Also start a counter for days after threshold
delay_counter = 0

#Vaccine efficacy delay
vacc_eff_delay = 14
#Set up a matrix with as many columns as days, and as many rows as there are districts
#District names are used as index
eff_delay_counter_Sf_mat = pd.DataFrame(np.zeros(shape = (len(gdf), vacc_eff_delay)), columns = ['Day' + str(i) for i in range(vacc_eff_delay)])
eff_delay_counter_Sf_mat = eff_delay_counter_Sf_mat.set_index(gdf['adm2'])

eff_delay_counter_HCW_mat = pd.DataFrame(np.zeros(shape = (len(gdf), vacc_eff_delay)), columns = ['Day' + str(i) for i in range(vacc_eff_delay)])
eff_delay_counter_HCW_mat = eff_delay_counter_HCW_mat.set_index(gdf['adm2'])

## Set up the epidemiological model

In [23]:
#Make a number of lambda functions corresponding to all the steps of the epidemiological model

#Vaccination of HCW
HCW_vaccination = lambda x: np.random.uniform(0, 1, x[11]) < vacc_rate_HCW
HCW_vaccination_effect = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0])

#Vaccination of family members
Sf_vaccination = lambda x: np.random.uniform(0, 1, x[3]) < ring_vac
Sf_vaccination_effect = np.array([0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#Exposure of susceptibles in the community
Sc_exposed = lambda x: np.random.uniform(0, 1, x[0]) < beta * x[5] / (x[0] + x[1] + x[2] + x[5] + x[10])
Sc_exposed_effect = np.array([-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#Exposure of vaccinated and protected in the household layer
V_exposed = lambda x: np.random.uniform(0, 1, x[10]) < (1 - phi) * beta * x[5] / (x[3] + x[4] + x[5] + x[2] + x[10])
V_exposed_effect = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0])

#Exposed in community become infectious
Ec_infectious = lambda x: np.random.uniform(0, 1, x[1]) < omega
Ec_infectious_effect = np.array([0, -1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

#Exposure of susceptibles in family layer
Sf_exposed = lambda x: np.random.uniform(0, 1, int(x[3])) < (beta / (tau * (np.sum(newI[district]) + newI_row[district][0]))) * x[5]
Sf_exposed_effect = np.array([0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

#Exposed in family layer become infectious
Ef_infectious = lambda x: np.random.uniform(0, 1, x[4]) < omega
Ef_infectious_effect = np.array([0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0])

#Infectious in family layer go to hospital
If_to_hospital = lambda x: np.random.uniform(0, 1, x[5]) < pH
If_to_hospital_effect = np.array([0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0])

#Infectious in family layer dies
If_death = lambda x: np.random.uniform(0, 1, x[5]) < alpha
If_death_effect = np.array([0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0])

#Infectious in family layer recovers
If_recovery = lambda x: np.random.uniform(0, 1, x[5]) < gamma
If_recovery_effect = np.array([0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0])

#Susceptible in hospital exposed
Sh_exposed = lambda x: np.random.uniform(0, 1, x[6]) < beta * x[8] / (x[6] + x[7] + x[8] + x[9] + x[11] + x[12])
Sh_exposed_effect = np.array([0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0])

#Non vaccinated HCW exposed
HCW_exposed = lambda x: np.random.uniform(0, 1, x[11]) < unsafe * beta * x[8] / (x[6] + x[7] + x[8] + x[9] + x[11] + x[12])
HCW_exposed_effect = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0])

#Vaccinated HCW exposed
HCWv_exposed = lambda x: np.random.uniform(0, 1, x[12]) < (1 - phi) * beta * x[8] / (x[6] + x[7] + x[8] + x[9] + x[11] + x[12])
HCWv_exposed_effect = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -1])

#Exposed in hospital to infectious
Eh_infectious = lambda x: np.random.uniform(0, 1, x[7]) < omega
Eh_infectious_effect = np.array([0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0])

#Infectious in hospital die
Ih_death = lambda x: np.random.uniform(0, 1, x[8]) < alpha
Ih_death_effect = np.array([0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0])

#Infectious in hospital recover
Ih_recovery = lambda x: np.random.uniform(0, 1, x[8]) < gamma
Ih_recovery_effect = np.array([0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0])

#Recovered in hospital return to community
Rh_to_community = lambda x: np.random.uniform(0, 1, x[9]) < pC
Rh_to_community_effect = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0])

## Set the initial conditions

In [40]:
#Initiate a counter for the number of days since last infected in the population
#Will remain at 0 while there are infected in the population
infect_counter = 0

#Initiate a dataframe for the number of new infected in the community per day
#Each district is a column
newI = pd.DataFrame(columns = gdf['adm2'])
newI['Days'] = []

#Also for the infected individuals create an object that will be used to populate the first row - with the initial infected
newI_row = np.zeros(shape = (1, len(newI.columns)))
newI_row = pd.DataFrame(newI_row, columns = newI.columns)

#Initiate a dataframe for the number of new infected in the hospital per day
#Each district is a column
newI_hosp = pd.DataFrame(columns = gdf['adm2'])
newI_hosp['Days'] = []

#Initiate a dataframe for the number of vaccine doses used per days
#Each district is a column
Vdoses= pd.DataFrame(columns = gdf['adm2'])
Vdoses['Days'] = []

#Initiate a dataframe for the numbers of new individuals added to the household layer
#Each district is a column
newSh = pd.DataFrame(columns = gdf['adm2'])
newSh['Days'] = []

#Also for the infected individuals create an object that will be used to populate the first row - with the initial infected
newSh_row = np.zeros(shape = (1, len(newSh.columns)))
newSh_row = pd.DataFrame(newSh_row, columns = newSh.columns)

#Initiate a counter for the days
t = [0]

#how many individuals to seed with (i.e. n=1 means a single index case)
n_seed = 1

#Create an object with the initial conditions
#It is set to have the entire population as susceptible
#In addition number of beds and number of physicians/nurses (HCWs) are based on data
#Data source: data.worldbank.org

epi_state_district = []

for district in gdf['adm2']:
    pop = gdf[gdf['adm2'] == district]['POPULATION'].astype(int).values[0]
    district_vals = [pop, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    if district == gdf['adm2'][emerg_loc]: #Take one susceptible and make it infected to seed the epidemic
        district_vals[5] = n_seed
        district_vals[3] = int(np.sum(tn.rvs(n_seed))) #Also move a random number of individuals to the family layer
        district_vals[0] = district_vals[0] - (district_vals[5] + district_vals[3])
        newI_row[district] = n_seed #Add the individual to the dataframe of new infections
        newI_row['Days'] = 0 #Also add the Days column to match the format
        newSh_row[district] = district_vals[3] #Add the individual to the dataframe of Sh
        newSh_row['Days'] = 0 #Also add the Days column to match the format

    epi_state_district.append(district_vals)

#Make it a pandas dataframe with obvious column names
epi_state_district = pd.DataFrame(epi_state_district,
                 columns=list(['Sc', 'Ec', 'Rc', 'Sf', 'Ef', 'If', 'Sh', 'Eh', 'Ih', 'Rh', 'V', 'HCW', 'HCWv']))

#Add the values in the HCW column
#HCWs data according to World Bank (2017)
#These are separated for India and Bangladesh
phys_bgd = 0.5 / 1000
phys_ind = 0.8 / 1000
nurse_bgd = 0.3 / 1000
nurse_ind = 2.1 / 1000

HCW_bgd = phys_bgd + nurse_bgd
HCW_ind = phys_ind + nurse_ind

#Calculate HCW numbers by adm2 based on population
epi_state_district['HCW'] = [int(gdf['POPULATION'].astype(float)[i] * HCW_ind) if gdf['ISO'][i] == 'IND' else int(gdf['POPULATION'].astype(float)[i] * HCW_bgd) for i in range(len(gdf)) ]

#Do the same for hospital beds
hosp_bgd = 0.8 / 4000
hosp_ind = 0.7 / 4000
epi_state_district['Sh'] = [int(gdf['POPULATION'].astype(float)[i] * hosp_ind) if gdf['ISO'][i] == 'IND' else int(gdf['POPULATION'].astype(float)[i] * hosp_bgd) for i in range(len(gdf)) ]

#Add a district column based on the geo dataframe
epi_state_district['district'] = gdf['adm2']

#Set the district name as index
epi_state_district = epi_state_district.set_index('district')

#Add the initial row to the new infections DataFrame
newI = pd.concat([newI, newI_row])
newSh = pd.concat([newSh, newSh_row])

## Run the simulation

In [41]:
#Start the simulation
#Time step is daily
#Set it to stop once the counter reaches min_time_after_epidemic
#i.e. when there hasn't been a case for long enough

while infect_counter < min_time_after_epidemic:

    #Make a deep copy of the state of epidemic at the beggining of the day in the loop
    epi_state_district_calc = epi_state_district.copy()

    #Create a pandas dataframe for the new infected individuals with the district names as columns
    newI_row = np.zeros(shape = (1, len(newI.columns)))
    newI_row = pd.DataFrame(newI_row, columns = newI.columns)

    #Create a pandas dataframe for the new infected individuals with the district names as columns
    newI_hosp_row = np.zeros(shape = (1, len(newI_hosp.columns)))
    newI_hosp_row = pd.DataFrame(newI_row, columns = newI_hosp.columns)

    #Create a pandas dataframe for the number of vaccine doses used, with the district names as columns
    Vdoses_row = np.zeros(shape = (1, len(Vdoses.columns)))
    Vdoses_row = pd.DataFrame(Vdoses_row, columns = Vdoses.columns)

    #Create a pandas dataframe for the new household susceptibles
    newSh_row = np.zeros(shape = (1, len(newSh.columns)))
    newSh_row = pd.DataFrame(newSh_row, columns = newSh.columns)

    #Loop over all the districts
    for district in gdf['adm2']:
        #Take the corresponding row for the current district for the vaccination delay
        eff_delay_counter_Sf = [int(i) for i in eff_delay_counter_Sf_mat.loc[district].to_list()]
        eff_delay_counter_HCW = [int(i) for i in eff_delay_counter_HCW_mat.loc[district].to_list()]

        #Take the row corresponding to the district
        epi_state_t = epi_state_district.loc[district].to_numpy()

        #Calculate the district specific rate of transfer from community to hospital
        pH = 1 / time_to_hospital.loc[district][0]

        #If there are enough cases, vaccinate
        if np.sum(newI[district]) > threshold:
            if delay_counter > vacc_delay:
                #Vaccination of HCW
                change_HCW_vaccination = HCW_vaccination(epi_state_t)
                epi_state_t = epi_state_t + np.einsum('j, i -> j', HCW_vaccination_effect, change_HCW_vaccination)

                new_protected_HCW = eff_delay_counter_HCW.pop()
                eff_delay_counter_HCW = list(np.append(np.sum(change_HCW_vaccination), eff_delay_counter_HCW))
                epi_state_t[12] += new_protected_HCW

                #Vaccination of family members
                #change_Sf_vaccination = Sf_vaccination(epi_state_t)
                #epi_state_t = epi_state_t + np.einsum('j, i -> j', Sf_vaccination_effect, change_Sf_vaccination)

                #new_protected_Sf = eff_delay_counter_Sf.pop()
                #eff_delay_counter_Sf = list(np.append(np.sum(change_Sf_vaccination), eff_delay_counter_Sf))
                #epi_state_t[10] += new_protected_Sf

                #Vaccination of susceptibles in the community - a.k.a. random vaccination
                #Only happens if enough time since beginning has happened for any contact tracing
                #Draw a number
                #if days_random_vac <= t[-1]:
                #    n_to_V = int(np.sum(tn_vac.rvs(int(newI[district][t[-1]-days_random_vac]))))
                    #Safeguard to avoid negative numbers
                #    if n_to_V > epi_state_t[0]:
                #        n_to_V = epi_state_t[0]
                #    epi_state_t = epi_state_t + np.array([-n_to_V, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
                #    eff_delay_counter_Sf[0] += n_to_V

                #Calculate the number of vaccine doses used
                #Vdoses_row[district] = n_to_V + np.sum(change_HCW_vaccination) + np.sum(change_Sf_vaccination)
                #Vdoses_row[district] =np.sum(change_HCW_vaccination) + np.sum(change_Sf_vaccination)
                Vdoses_row[district] =np.sum(change_HCW_vaccination)

                #Add one to delay delay_counter
                delay_counter += 1

            else:
                #Add 1 to the counter
                delay_counter += 1
                #Add 0 to the number of vaccine doses
                Vdoses_row[district] = 0

        else: #if there are not enough cases, there is no vaccination - set vaccine doses used to 0
            Vdoses_row[district] = 0

        #Calculate if there are exposed or infectious individuals in the pop
        crit = epi_state_district.loc[district]['Ec'] + epi_state_district.loc[district]['Ef'] + epi_state_district.loc[district]['If'] + epi_state_district.loc[district]['Eh'] + epi_state_district.loc[district]['Ih']

        if crit > 0: #If there are infected/infectious full model
            #Exposure of susceptibles in the community
            change_Sc_exposed = Sc_exposed(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', Sc_exposed_effect, change_Sc_exposed)

            #Exposure of vaccinated
            if epi_state_t[10] > 0:
                change_V_exposed = V_exposed(epi_state_t)
                epi_state_t = epi_state_t + np.einsum('j, i -> j', V_exposed_effect, change_V_exposed)

            #Get the list of districts with exchanges
            adm1 = gdf[gdf['adm2'] == district]['NAME_1'].values[0]
            subset_adm2 = gdf[gdf['NAME_1'] == adm1]['adm2']
            key_list = list(w[gdf[gdf['adm2'] == district].index.values[0]].keys())
            list_adm2 = list(subset_adm2)
            list_adm2.extend([gdf['adm2'].iloc[key] for key in key_list])
            list_loc = list(np.unique(list_adm2))

            #Find districts with current infected - others present no risk
            infect_risk = []
            for loc in list_loc:
                if epi_state_district_calc['If'][loc] > 0:
                    infect_risk.append(loc)

            #Calculate new infections based on travels
            new_Sh_infect = 0
            new_V_infect = 0
            for risk_loc in infect_risk:
                if risk_loc != district:
                    n_Sh_travelers = int(connectivity[district][risk_loc] * epi_state_t[0] / (epi_state_t[0] + epi_state_t[1] + epi_state_t[2] + epi_state_t[5] + epi_state_t[10]))
                    if n_Sh_travelers > epi_state_t[0]:
                        n_Sh_travelers = epi_state_t[0]

                    n_V_travelers = int(connectivity[district][risk_loc] * epi_state_t[10] / (epi_state_t[0] + epi_state_t[1] + epi_state_t[2] + epi_state_t[5] + epi_state_t[10]))
                    if n_V_travelers > epi_state_t[10]:
                        n_V_travelers = epi_state_t[10]

                    new_infect_from_Sh_travelers = np.sum(np.random.uniform(0, 1, n_Sh_travelers) < beta * epi_state_district_calc['If'].loc[risk_loc] / (epi_state_district_calc['Sc'].loc[risk_loc] + epi_state_district_calc['Ec'].loc[risk_loc] + epi_state_district_calc['If'].loc[risk_loc] + epi_state_district_calc['Rc'].loc[risk_loc]))
                    new_Sh_infect += new_infect_from_Sh_travelers
                    new_infect_from_V_travelers = np.sum(np.random.uniform(0, 1, n_V_travelers) < beta * epi_state_district_calc['If'].loc[risk_loc] / (epi_state_district_calc['Sc'].loc[risk_loc] + epi_state_district_calc['Ec'].loc[risk_loc] + epi_state_district_calc['If'].loc[risk_loc] + epi_state_district_calc['Rc'].loc[risk_loc]))
                    new_V_infect += new_infect_from_V_travelers

                    n_infected_visitors = np.random.binomial(connectivity[risk_loc][district], epi_state_district_calc['If'][risk_loc] / (epi_state_district_calc['Sc'][risk_loc] + epi_state_district_calc['Ec'][risk_loc] + epi_state_district_calc['Rc'][risk_loc] + epi_state_district_calc['If'][risk_loc]), 1)
                    new_Sh_infect_from_visitors = np.sum(np.random.uniform(0, 1, epi_state_t[0]) < beta * n_infected_visitors / (epi_state_t[0] + epi_state_t[1] + epi_state_t[2] + epi_state_t[5] + epi_state_t[10]))
                    new_Sh_infect += new_Sh_infect_from_visitors
                    new_V_infect_from_visitors = np.sum(np.random.uniform(0, 1, epi_state_t[10]) < beta * n_infected_visitors / (epi_state_t[0] + epi_state_t[1] + epi_state_t[2] + epi_state_t[5] + epi_state_t[10]))
                    new_V_infect += new_V_infect_from_visitors

            if new_Sh_infect > epi_state_t[0]:
                new_Sh_infect = epi_state_t[0]
            if new_V_infect > epi_state_t[10]:
                new_V_infect = epi_state_t[10]
            epi_state_t = epi_state_t + np.array([-new_Sh_infect, new_Sh_infect + new_V_infect, 0, 0, 0, 0, 0, 0, 0, 0, -new_V_infect, 0, 0])

            #Exposed in community become infectious
            change_Ec_infectious = Ec_infectious(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', Ec_infectious_effect, change_Ec_infectious)

            #Update newI row
            newI_row[district] += np.sum(change_Ec_infectious)

            #Exposure of susceptibles in family layer
            if (np.sum(newI[district]) + newI_row[district][0]) > 0:
                change_Sf_exposed = Sf_exposed(epi_state_t)
                epi_state_t = epi_state_t + np.einsum('j, i -> j', Sf_exposed_effect, change_Sf_exposed)

            #Draw a number of individuals to add to the family layer based on the number of new infections
            n_to_Sf = int(np.sum(tn.rvs(np.sum(change_Ec_infectious) + np.sum(change_Sf_exposed))))
            if n_to_Sf > epi_state_t[0]:
                n_to_Sf = epi_state_t[0]
            epi_state_t = epi_state_t + np.array([-n_to_Sf, 0, 0, n_to_Sf, 0, 0, 0, 0, 0, 0, 0, 0, 0])
            newSh_row[district] = n_to_Sf

            #Exposure of vaccinated and unprotected in the family layer
            vac_exposed = 0
            if np.sum(newI[district] > 0): #Only happens if there are infected in the pop - otherwise division by 0 when first exposed in a new district
                for i in range(len(eff_delay_counter_Sf)):
                    vac_exposed_day = np.sum(np.random.uniform(0, 1, eff_delay_counter_Sf[i]) < (beta / (tau * (np.sum(newI[district]) + newI_row[district][0]))) * epi_state_t[5])
                    eff_delay_counter_Sf[i] -= vac_exposed_day
                    vac_exposed += vac_exposed_day

            epi_state_t[4] +=  vac_exposed

            #Exposed in family layer become infectious
            change_Ef_infectious = Ef_infectious(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', Ef_infectious_effect, change_Ef_infectious)

            #Update newI row
            newI_row[district] += np.sum(change_Ef_infectious)

            #Infectious in family layer go to hospital
            change_If_to_hospital = If_to_hospital(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', If_to_hospital_effect, change_If_to_hospital)

            #Infectious in family layer dies
            change_If_death = If_death(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', If_death_effect, change_If_death)

            #Infectious in family layer recovers
            change_If_recovery = If_recovery(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', If_recovery_effect, change_If_recovery)

            #Susceptible in hospital exposed
            change_Sh_exposed = Sh_exposed(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', Sh_exposed_effect, change_Sh_exposed)

            #Non vaccinated HCW exposed
            change_HCW_exposed = HCW_exposed(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', HCW_exposed_effect, change_HCW_exposed)

            #Exposure of vaccinated and unprotected HCws
            vac_exposed_HCW = 0
            for i in range(len(eff_delay_counter_HCW)):
                vac_exposed_day_HCW = np.sum(np.random.uniform(0, 1, eff_delay_counter_HCW[i]) < unsafe * beta * epi_state_t[8] / (epi_state_t[6] + epi_state_t[7] + epi_state_t[8] + epi_state_t[9] + epi_state_t[11] + epi_state_t[12]))
                eff_delay_counter_HCW[i] -= vac_exposed_day_HCW
                vac_exposed_HCW += vac_exposed_day_HCW
            epi_state_t[7] +=  vac_exposed_HCW

            #Vaccinated HCW exposed
            change_HCWv_exposed = HCWv_exposed(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', HCWv_exposed_effect, change_HCWv_exposed)

            #Exposed in hospital to infectious
            change_Eh_infectious = Eh_infectious(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', Eh_infectious_effect, change_Eh_infectious)

            #Update newI in hospital row
            newI_hosp_row[district] += np.sum(change_Eh_infectious)

            #Infectious in hospital die
            change_Ih_death = Ih_death(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', Ih_death_effect, change_Ih_death)

            #Infectious in hospital recover
            change_Ih_recovery = Ih_recovery(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', Ih_recovery_effect, change_Ih_recovery)

            #Recovered in hospital return to community
            change_Rh_to_community = Rh_to_community(epi_state_t)
            epi_state_t = epi_state_t + np.einsum('j, i -> j', Rh_to_community_effect, change_Rh_to_community)

        else: #No infected or infectious - they can get infected from elsewhere
            #Get the list of districts with exchanges
            adm1 = gdf[gdf['adm2'] == district]['NAME_1'].values[0]
            subset_adm2 = gdf[gdf['NAME_1'] == adm1]['adm2']
            key_list = list(w[gdf[gdf['adm2'] == district].index.values[0]].keys())
            list_adm2 = list(subset_adm2)
            list_adm2.extend([gdf['adm2'].iloc[key] for key in key_list])
            list_loc = list(np.unique(list_adm2))

            #Find districts with current infected - others present no risk
            infect_risk = []
            for loc in list_loc:
                if epi_state_district_calc['If'][loc] > 0:
                    infect_risk.append(loc)

            #Calculate new infections based on travels
            new_Sh_infect = 0
            new_V_infect = 0
            for risk_loc in infect_risk:
                n_Sh_travelers = int(connectivity[district][risk_loc] * epi_state_t[0] / (epi_state_t[0] + epi_state_t[1] + epi_state_t[2] + epi_state_t[5] + epi_state_t[10]))
                if n_Sh_travelers > epi_state_t[0]:
                    n_Sh_travelers = epi_state_t[0]

                n_V_travelers = int(connectivity[district][risk_loc] * epi_state_t[10] / (epi_state_t[0] + epi_state_t[1] + epi_state_t[2] + epi_state_t[5] + epi_state_t[10]))
                if n_V_travelers > epi_state_t[10]:
                    n_V_travelers = epi_state_t[10]

                new_infect_from_Sh_travelers = np.sum(np.random.uniform(0, 1, n_Sh_travelers) < beta * epi_state_district_calc['If'].loc[risk_loc] / (epi_state_district_calc['Sc'].loc[risk_loc] + epi_state_district_calc['Ec'].loc[risk_loc] + epi_state_district_calc['If'].loc[risk_loc] + epi_state_district_calc['Rc'].loc[risk_loc]))
                new_Sh_infect += new_infect_from_Sh_travelers
                new_infect_from_V_travelers = np.sum(np.random.uniform(0, 1, n_V_travelers) < beta * epi_state_district_calc['If'].loc[risk_loc] / (epi_state_district_calc['Sc'].loc[risk_loc] + epi_state_district_calc['Ec'].loc[risk_loc] + epi_state_district_calc['If'].loc[risk_loc] + epi_state_district_calc['Rc'].loc[risk_loc]))
                new_V_infect += new_infect_from_V_travelers

                n_infected_visitors = np.random.binomial(connectivity[risk_loc][district], epi_state_district_calc['If'][risk_loc] / (epi_state_district_calc['Sc'][risk_loc] + epi_state_district_calc['Ec'][risk_loc] + epi_state_district_calc['Rc'][risk_loc] + epi_state_district_calc['If'][risk_loc]), 1)
                new_Sh_infect_from_visitors = np.sum(np.random.uniform(0, 1, epi_state_t[0]) < beta * n_infected_visitors / (epi_state_t[0] + epi_state_t[1] + epi_state_t[2] + epi_state_t[5] + epi_state_t[10]))
                new_Sh_infect += new_Sh_infect_from_visitors
                new_V_infect_from_visitors = np.sum(np.random.uniform(0, 1, epi_state_t[10]) < beta * n_infected_visitors / (epi_state_t[0] + epi_state_t[1] + epi_state_t[2] + epi_state_t[5] + epi_state_t[10]))
                new_V_infect += new_V_infect_from_visitors

            if new_Sh_infect > epi_state_t[0]:
                new_Sh_infect = epi_state_t[0]
            if new_V_infect > epi_state_t[10]:
                new_V_infect = epi_state_t[10]
            epi_state_t = epi_state_t + np.array([-new_Sh_infect, new_Sh_infect + new_V_infect, 0, 0, 0, 0, 0, 0, 0, 0, -new_V_infect, 0, 0])

            if epi_state_district.loc[district]['Rh'] > 0: #If there are recovered left in hospital
                #Recovered in hospital return to community
                change_Rh_to_community = Rh_to_community(epi_state_t)
                epi_state_t = epi_state_t + np.einsum('j, i -> j', Rh_to_community_effect, change_Rh_to_community)

        #Update epi_state_district with the changes in the current district
        for i in range(len(epi_state_t)):
            epi_state_district.loc[district][i] = epi_state_t[i]

        #Update the vaccination delay dataframes
        eff_delay_counter_Sf_mat.loc[district] = eff_delay_counter_Sf
        eff_delay_counter_HCW_mat.loc[district] = eff_delay_counter_HCW

    #Add one to the day
    t.append(t[-1] + 1)

    if t[-1] == 365:
        print("Maximum time reached - ending simulation")
        break


    #If there are no exposed or infected in are no individuals in the population, add 1 to the counter
    if np.sum(epi_state_district['If'])+ np.sum(epi_state_district['Ef']) + np.sum(epi_state_district['Eh']) + np.sum(epi_state_district['Ih']) == 0:
        infect_counter += 1
    else:
        infect_counter = 0

    #Add the day to the corresponding column in the new infection and the vaccine doses
    newI_row['Days'] = t[-1]
    newI_hosp_row['Days'] = t[-1]
    Vdoses_row['Days'] = t[-1]
    newSh_row['Days'] = t[-1]

    #Add the daily row to the objects for vaccine doses and new infections
    Vdoses = pd.concat([Vdoses, Vdoses_row])
    Vdoses = Vdoses.reset_index(drop = True)
    newI = pd.concat([newI, newI_row])
    newI = newI.reset_index(drop = True)
    newI_hosp = pd.concat([newI_hosp, newI_hosp_row])
    newI_hosp = newI_hosp.reset_index(drop = True)
    newSh = pd.concat([newSh, newSh_row])
    newSh = newSh.reset_index(drop = True)

1
2
3
4
5
6
7
8
9
10
Maximum time reached - ending simulation
